In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import mean_squared_error as mse
import lightgbm
from lightgbm import LGBMRegressor

In [6]:
train_df = pd.read_csv("../data/Train.csv")
test_df = pd.read_csv("../data/Test.csv")
ss = pd.read_csv("../data/SampleSubmission.csv")

In [7]:
from datetime import datetime
def fix_time(old_time):
    new_time=datetime.fromtimestamp(old_time)
    return new_time


In [8]:
train_df["Date"]=train_df.timestamp.apply(fix_time)
test_df["Date"]=test_df.timestamp.apply(fix_time)

In [9]:
train_df["Date"]

0       2021-06-22 14:03:28
1       2021-06-22 15:34:25
2       2021-06-22 15:24:31
3       2021-06-24 14:41:06
4       2021-06-24 14:53:18
                ...        
34269   2021-06-22 10:27:41
34270   2021-06-22 09:19:53
34271   2021-06-22 16:52:15
34272   2021-06-24 14:54:19
34273   2021-06-22 16:42:51
Name: Date, Length: 34274, dtype: datetime64[ns]

In [10]:
test_df

,id,timestamp,device,PCell_RSRP_max,PCell_RSRQ_max,PCell_RSSI_max,PCell_SNR_max,PCell_Downlink_Num_RBs,PCell_Downlink_Average_MCS,PCell_Downlink_bandwidth_MHz,...,dewPoint,humidity,pressure,windSpeed,cloudCover,uvIndex,visibility,Traffic Jam Factor,area,Date
0,Id_ln0e0hfrgx,1624374179,pc2,-97.121875,-12.923750,-87.554375,13.293,39276.0,24.0,20,...,14.55,0.59,1013.6,2.27,0.83,3.0,16.093,3.07702,Avenue,2021-06-22 16:02:59
1,Id_svf7nz9fxv,1624366264,pc4,-65.202500,-9.944375,-35.501875,21.823,39566.0,25.0,20,...,13.92,0.62,1013.6,2.42,0.95,4.0,16.093,0.66000,Park,2021-06-22 13:51:04
2,Id_ww2mh07gwj,1624375795,pc2,-79.298125,-17.468750,-74.823750,24.114,41900.0,27.0,20,...,14.65,0.60,1013.5,2.26,0.85,3.0,16.093,2.49330,Park,2021-06-22 16:29:55
3,Id_v88r4y03ww,1624374065,pc2,-110.128125,-13.513750,-92.566875,6.282,76310.0,18.0,20,...,14.45,0.59,1013.6,2.27,0.82,3.0,16.093,5.74454,Highway,2021-06-22 16:01:05
4,Id_2u4y4kzglh,1624374361,pc2,-117.451250,-18.753125,-93.048125,-5.767,44887.0,7.0,20,...,14.55,0.59,1013.6,2.27,0.83,3.0,16.093,3.48651,Avenue,2021-06-22 16:06:01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18238,Id_thgt73179f,1624370207,pc2,-81.506250,-9.556875,-78.813125,24.005,49881.0,28.0,20,...,14.77,0.62,1013.8,2.12,0.84,3.0,16.093,1.82751,Park,2021-06-22 14:56:47
18239,Id_r6wd5ofvtt,1624371137,pc2,-84.858750,-12.547500,-83.305000,11.985,29323.0,22.0,20,...,14.69,0.62,1013.9,2.12,0.83,3.0,16.093,3.14159,Park,2021-06-22 15:12:17
18240,Id_p6sqas9rgd,1624376766,pc2,-82.406875,-12.343125,-79.631875,12.719,37837.0,21.0,20,...,14.78,0.57,1013.7,2.19,0.76,2.0,16.093,1.57307,Residential,2021-06-22 16:46:06
18241,Id_yre3gwygul,1624353986,pc4,-95.212500,-10.181875,-65.595625,11.800,47277.0,15.0,20,...,13.92,0.73,1012.2,4.23,0.97,4.0,16.093,0.51020,Park,2021-06-22 10:26:26


In [11]:
target = "target"

In [12]:
train_df["xx"]=train_df.groupby(target)[target].transform("count")
train_df=train_df[train_df["xx"]>1]
train_df=train_df.drop("xx",axis=1)


In [13]:
validation=0

In [14]:
train_df

,id,timestamp,device,PCell_RSRP_max,PCell_RSRQ_max,PCell_RSSI_max,PCell_SNR_max,PCell_Downlink_Num_RBs,PCell_Downlink_Average_MCS,PCell_Downlink_bandwidth_MHz,...,humidity,pressure,windSpeed,cloudCover,uvIndex,visibility,Traffic Jam Factor,area,target,Date
0,Id_qt2qn56050,1624367008,pc1,-84.498750,-14.586875,-49.855625,16.457,35076.0,23.0,20.0,...,0.62,1013.6,2.39,0.95,4.0,16.093,0.38979,Residential,51500000.0,2021-06-22 14:03:28
1,Id_f1zf07cwb6,1624372465,pc3,-86.818125,-11.982500,-54.838125,16.674,42704.0,24.0,20.0,...,0.59,1013.5,2.29,0.82,3.0,16.093,3.78322,Residential,50300000.0,2021-06-22 15:34:25
2,Id_uoxdz7di5b,1624371871,pc3,-101.916250,-13.257500,-71.241250,9.392,36017.0,22.0,20.0,...,0.61,1013.8,2.13,0.82,3.0,16.093,2.08425,Residential,16500000.0,2021-06-22 15:24:31
3,Id_hzn8nj29gu,1624542066,pc1,-95.598750,-14.696250,-61.660000,10.726,95955.0,17.0,20.0,...,0.71,1017.9,1.97,0.99,3.0,16.093,8.80702,Avenue,46400000.0,2021-06-24 14:41:06
4,Id_2kfrccrfiw,1624542798,pc1,-97.735000,-14.867500,-63.170625,10.889,83062.0,19.0,20.0,...,0.71,1017.9,1.97,0.99,3.0,16.093,2.21843,Park,50200000.0,2021-06-24 14:53:18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34269,Id_lc1vv34d0r,1624354061,pc1,-72.789375,-10.436875,-43.340625,6.342,96320.0,16.0,20.0,...,0.73,1012.2,4.23,0.97,4.0,16.093,2.66218,Park,78800000.0,2021-06-22 10:27:41
34270,Id_qywxdy2q50,1624349993,pc3,-86.219375,-12.911875,-56.473750,13.201,45069.0,27.0,20.0,...,0.77,1011.9,4.07,0.97,3.0,16.093,2.08918,Park,25000000.0,2021-06-22 09:19:53
34271,Id_5ll45hn5yd,1624377135,pc3,-85.964375,-13.278750,-51.610000,11.295,33942.0,19.0,20.0,...,0.57,1013.7,2.22,0.76,2.0,16.093,1.94504,Residential,36300000.0,2021-06-22 16:52:15
34272,Id_7uz4vnrnzu,1624542859,pc1,-92.546250,-14.460625,-58.441875,13.096,87065.0,20.0,20.0,...,0.71,1017.9,1.97,0.99,3.0,16.093,1.24398,Park,52800000.0,2021-06-24 14:54:19


In [15]:
train_df[target].value_counts()

101000000.0    103
141000000.0     94
102000000.0     88
107000000.0     87
110000000.0     87
              ... 
2120000.0        2
9730000.0        2
5150000.0        2
8360000.0        2
9970000.0        2
Name: target, Length: 1481, dtype: int64

In [16]:
train_df["device"].unique()

array(['pc1', 'pc3'], dtype=object)

In [17]:
test_df["device"].unique()

array(['pc2', 'pc4'], dtype=object)

In [18]:
train_df["timestamp"].value_counts()

1624546748    2
1624551351    2
1624547690    2
1624547622    2
1624551307    2
             ..
1624367902    1
1624542101    1
1624369420    1
1624368572    1
1624376571    1
Name: timestamp, Length: 27197, dtype: int64

In [19]:
train_df.corr().abs()[target].sort_values(ascending=False)

target                          1.000000
SCell_SNR_max                   0.642068
SCell_Downlink_Average_MCS      0.627163
PCell_SNR_max                   0.572684
PCell_RSSI_max                  0.503193
SCell_RSRP_max                  0.503059
SCell_Downlink_Num_RBs          0.483362
PCell_RSRP_max                  0.472979
SCell_RSSI_max                  0.452446
PCell_Downlink_Average_MCS      0.444546
operator                        0.431136
PCell_Cell_Identity             0.413958
PCell_Downlink_Num_RBs          0.403038
SCell_RSRQ_max                  0.289486
PCell_Downlink_bandwidth_MHz    0.251947
temperature                     0.249340
apparentTemperature             0.249340
cloudCover                      0.248666
dewPoint                        0.246258
humidity                        0.230337
timestamp                       0.201864
PCell_RSRQ_max                  0.191119
pressure                        0.181916
precipProbability               0.180588
SCell_Cell_Ident

In [20]:
test_df["area"]=test_df["area"].replace({'Avenue':0, 'Park':1, 'Highway':2, 'Residential':3, 'Tunnel':4})
train_df["area"]=train_df["area"].replace({'Avenue':0, 'Park':1, 'Highway':2, 'Residential':3, 'Tunnel':4})
test_df.device=test_df.device.str[-1].astype("int")
train_df.device=train_df.device.str[-1].astype("int")

In [21]:
if validation==0:
    test_df[target]=np.nan
    test_df=test_df[train_df.columns]
    train_df=train_df.append(test_df)
else:
    test_df=train_df[train_df.device==3]
    ss = test_df[ss.columns]
    train_df=train_df[train_df.device==1]
    test_df[target]=np.nan
    test_df=test_df[train_df.columns]
    train_df=train_df.append(test_df)

In [22]:
ss

,id,target
0,Id_hg4i94kfbg,0.0
1,Id_j1afeth58t,0.0
2,Id_bahkipht8j,0.0
3,Id_t1iry5orn0,0.0
4,Id_dnlh2w774v,0.0
...,...,...
18238,Id_t4ukpeaxa5,0.0
18239,Id_dallii8q6q,0.0
18240,Id_p0neksuxwp,0.0
18241,Id_s2yku2tuvb,0.0


In [23]:
train_df

,id,timestamp,device,PCell_RSRP_max,PCell_RSRQ_max,PCell_RSSI_max,PCell_SNR_max,PCell_Downlink_Num_RBs,PCell_Downlink_Average_MCS,PCell_Downlink_bandwidth_MHz,...,humidity,pressure,windSpeed,cloudCover,uvIndex,visibility,Traffic Jam Factor,area,target,Date
0,Id_qt2qn56050,1624367008,1,-84.498750,-14.586875,-49.855625,16.457,35076.0,23.0,20.0,...,0.62,1013.6,2.39,0.95,4.0,16.093,0.38979,3,51500000.0,2021-06-22 14:03:28
1,Id_f1zf07cwb6,1624372465,3,-86.818125,-11.982500,-54.838125,16.674,42704.0,24.0,20.0,...,0.59,1013.5,2.29,0.82,3.0,16.093,3.78322,3,50300000.0,2021-06-22 15:34:25
2,Id_uoxdz7di5b,1624371871,3,-101.916250,-13.257500,-71.241250,9.392,36017.0,22.0,20.0,...,0.61,1013.8,2.13,0.82,3.0,16.093,2.08425,3,16500000.0,2021-06-22 15:24:31
3,Id_hzn8nj29gu,1624542066,1,-95.598750,-14.696250,-61.660000,10.726,95955.0,17.0,20.0,...,0.71,1017.9,1.97,0.99,3.0,16.093,8.80702,0,46400000.0,2021-06-24 14:41:06
4,Id_2kfrccrfiw,1624542798,1,-97.735000,-14.867500,-63.170625,10.889,83062.0,19.0,20.0,...,0.71,1017.9,1.97,0.99,3.0,16.093,2.21843,1,50200000.0,2021-06-24 14:53:18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18238,Id_thgt73179f,1624370207,2,-81.506250,-9.556875,-78.813125,24.005,49881.0,28.0,20.0,...,0.62,1013.8,2.12,0.84,3.0,16.093,1.82751,1,NaN,2021-06-22 14:56:47
18239,Id_r6wd5ofvtt,1624371137,2,-84.858750,-12.547500,-83.305000,11.985,29323.0,22.0,20.0,...,0.62,1013.9,2.12,0.83,3.0,16.093,3.14159,1,NaN,2021-06-22 15:12:17
18240,Id_p6sqas9rgd,1624376766,2,-82.406875,-12.343125,-79.631875,12.719,37837.0,21.0,20.0,...,0.57,1013.7,2.19,0.76,2.0,16.093,1.57307,3,NaN,2021-06-22 16:46:06
18241,Id_yre3gwygul,1624353986,4,-95.212500,-10.181875,-65.595625,11.800,47277.0,15.0,20.0,...,0.73,1012.2,4.23,0.97,4.0,16.093,0.51020,1,NaN,2021-06-22 10:26:26


In [24]:
#train_df["year"]=train_df["Date"].astype(str).str[:4].astype("int")
#train_df["month"]=train_df["Date"].astype(str).str[5:7].astype("int")


In [25]:
#train_df["day"]=train_df["Date"].astype(str).str[8:10].astype("int")
train_df["hour"]=train_df["Date"].astype(str).str[11:13].astype("int")


In [26]:
#train_df["minute"]=train_df["Date"].astype(str).str[14:16].astype("int")
#train_df["sec"]=train_df["Date"].astype(str).str[17:19].astype("int")


In [27]:
del test_df

In [28]:
drop_cols=['id','device', 'target','Date']
cols=list(train_df.drop(drop_cols,axis=1).columns)
for a, col1 in enumerate(cols):
    for b, col2 in enumerate(cols):
        if a<b:
            train_df[col1+col2]=train_df[col1]*train_df[col2]
train_df["area_mean"] = train_df.groupby("PCell_Cell_Identity")[target].transform("mean")

test_df=train_df[train_df[target].isna()]
train_df=train_df[train_df[target].notna()]
train_cols=list(train_df.drop(drop_cols,axis=1).columns)
train_df=train_df.replace({np.inf:np.nan,(-1*np.inf):np.nan})
test_df=test_df.replace({np.inf:np.nan,(-1*np.inf):np.nan})
my_model=LGBMRegressor(objective="tweedie")#CatBoostRegressor(silent=True,learning_rate=0.1, n_estimators=10)
my_model.fit(train_df[train_cols],train_df[target])
predictions = my_model.predict(test_df[train_cols])
submission = pd.DataFrame(data=test_df["id"], columns = ["id"])
submission["target"] = predictions


OSError: Cannot save file into a non-existent directory: 'predictions'

In [29]:
if validation==0:
    df1=pd.merge(ss[['id']],submission,on='id',how='left')
    df1.to_csv('../predictions/prdiction_1.csv',index=False)
else:
    df1=pd.merge(ss,submission,on='id',how='left')
    print((mse(df1["target_x"],df1["target_y"])**0.5)/10**6)
df1

,id,target
0,Id_hg4i94kfbg,2.712542e+07
1,Id_j1afeth58t,2.668849e+07
2,Id_bahkipht8j,2.561986e+07
3,Id_t1iry5orn0,2.558827e+07
4,Id_dnlh2w774v,2.589420e+07
...,...,...
18238,Id_t4ukpeaxa5,5.013669e+07
18239,Id_dallii8q6q,4.640289e+07
18240,Id_p0neksuxwp,1.048030e+08
18241,Id_s2yku2tuvb,1.007666e+08
